<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%83%81%EC%9E%A5%ED%9A%8C%EC%82%AC_%EB%B6%84%EA%B8%B0_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.0.1
    Uninstalling termcolor-3.0.1:
      Successfully uninstalled termcolor-3.0.1


In [6]:
!pip install pandas openpyxl

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
import requests
import pandas as pd
import time

API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

file_path = "/content/drive/MyDrive/Working/data/list_by_KIND.xlsx"
corporations_df = pd.read_excel(file_path, sheet_name="lists", dtype={'code': str},engine='openpyxl')

exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 연도 및 분기 보고서 코드 리스트
report_periods = [
    ("2023", "11013", "1분기"),
    ("2023", "11012", "반기"),
    ("2023", "11014", "3분기"),
]

all_data = []

for year, reprt_code, label in report_periods:
    print(f"\n📥 {year}년 {label} 보고서 수집 시작\n")

    for _, row in corporations_df.iterrows():
        corp_code = row['corp_code']
        corp_name = row.get('name', '알 수 없음')

        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': reprt_code,
            'fs_div': 'OFS'  # 개별 재무제표
        }

        try:
            response = requests.get(FS_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get('status') == '000' and 'list' in data:
                unique_accounts = set()
                first_appearance_accounts = []

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')

                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    if any(account_name.startswith(cap) for cap in capital_related_accounts):
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            all_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": year,
                                "분기": label,
                                "계정과목명": account_name,
                                "account_id": account_id,
                                "금액": item.get('thstrm_amount', '데이터 없음'),
                                "전기 금액": item.get('frmtrm_amount', '데이터 없음'),
                                "전전기 금액": item.get('lfy_amount', '데이터 없음')
                            })
                    else:
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            all_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": year,
                                "분기": label,
                                "계정과목명": account_name,
                                "account_id": account_id,
                                "금액": item.get('thstrm_amount', '데이터 없음'),
                                "전기 금액": item.get('frmtrm_amount', '데이터 없음'),
                                "전전기 금액": item.get('lfy_amount', '데이터 없음')
                            })

                print(f"✅ {corp_name}({corp_code}) - {label} 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - {label} 재무 데이터 없음")
        except requests.exceptions.RequestException as e:
            print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

        time.sleep(1)

# 결과 저장
df_q = pd.DataFrame(all_data)
output_excel = "기업_재무제표_분기별_2023년_L2.xlsx"
df_q.to_excel(output_excel, index=False)

print(f"\n📦 데이터 저장 완료: {output_excel}")



📥 2023년 1분기 보고서 수집 시작

⚠️ 씨케이솔루션(480370) - 1분기 재무 데이터 없음
⚠️ LG씨엔에스(64400) - 1분기 재무 데이터 없음
⚠️ GS피앤엘(499790) - 1분기 재무 데이터 없음
⚠️ 엠앤씨솔루션(484870) - 1분기 재무 데이터 없음
⚠️ 더본코리아(475560) - 1분기 재무 데이터 없음
⚠️ 한화비전(489790) - 1분기 재무 데이터 없음
⚠️ 전진건설로봇(79900) - 1분기 재무 데이터 없음
⚠️ HS효성(487570) - 1분기 재무 데이터 없음
⚠️ 산일전기(62040) - 1분기 재무 데이터 없음
⚠️ 시프트업(462870) - 1분기 재무 데이터 없음
⚠️ 파라다이스(34230) - 1분기 재무 데이터 없음
⚠️ HD현대마린솔루션(443060) - 1분기 재무 데이터 없음
⚠️ SK이터닉스(475150) - 1분기 재무 데이터 없음
⚠️ 에이피알(278470) - 1분기 재무 데이터 없음
⚠️ 엘앤에프(66970) - 1분기 재무 데이터 없음
⚠️ 포스코DX(22100) - 1분기 재무 데이터 없음
⚠️ DS단석(17860) - 1분기 재무 데이터 없음
⚠️ 동인기연(111380) - 1분기 재무 데이터 없음


KeyboardInterrupt: 

In [22]:
import requests
import pandas as pd
import time

API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 여기에 API 키를 입력하세요
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

file_path = "/content/drive/MyDrive/Working/data/Listed_com_fss_test.xlsx"
corporations_df = pd.read_excel(file_path, sheet_name="lists", dtype={'corp_code': str}, engine='openpyxl')

target_accounts = [
    "자산총계", "유동자산", "비유동자산", "부채총계", "유동부채", "비유동부채", "자본총계",
    "매출액", "매출원가", "판매비와관리비", "영업이익(보고서기재)", "법인세비용차감전순이익", "법인세비용", "당기순이익"
]

report_periods = [
    ("2023", "11013", "1분기"),
    ("2023", "11012", "반기"),
    ("2023", "11014", "3분기"),
]

all_data = []

for year, reprt_code, label in report_periods:
    print(f"\n📥 {year}년 {label} 보고서 수집 시작\n")

    for _, row in corporations_df.iterrows():
        corp_code = row['corp_code']
        corp_name = row.get('name', '알 수 없음')

        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': reprt_code,
            'fs_div': 'OFS'  # 개별 재무제표
        }

        try:
            response = requests.get(FS_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get('status') == '000' and 'list' in data:
                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')

                    if account_name in target_accounts:
                        all_data.append({
                            "기업 이름": corp_name,
                            "기업 코드": corp_code,
                            "연도": year,
                            "분기": label,
                            "계정과목명": account_name,
                            "account_id": account_id,
                            "금액": item.get('thstrm_amount', '데이터 없음'),
                            "전기 금액": item.get('frmtrm_amount', '데이터 없음'),
                            "전전기 금액": item.get('lfy_amount', '데이터 없음')
                        })

                print(f"✅ {corp_name}({corp_code}) - {label} 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - {label} 재무 데이터 없음")
        except requests.exceptions.RequestException as e:
            print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

        time.sleep(1)

# 결과 저장
df_q = pd.DataFrame(all_data)
output_excel = "기업_재무제표_분기별_2023년_L2.xlsx"
df_q.to_excel(output_excel, index=False)

print(f"\n📦 데이터 저장 완료: {output_excel}")


📥 2023년 1분기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 1분기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 1분기 수집 완료
⚠️ 주식회사 엘지씨엔에스(00139834) - 1분기 재무 데이터 없음
✅ 한국철강 주식회사(00687711) - 1분기 수집 완료
✅ 대원강업 주식회사(00111847) - 1분기 수집 완료
✅ 키스코홀딩스 주식회사(00159740) - 1분기 수집 완료
✅ (주)팜젠사이언스(00135050) - 1분기 수집 완료
✅ (주)지니뮤직(00223762) - 1분기 수집 완료
✅ 새론오토모티브(주)(00305668) - 1분기 수집 완료
✅ 아이에스동서(주)(00115977) - 1분기 수집 완료
✅ 한온시스템주식회사(00161125) - 1분기 수집 완료
✅ 글로벌텍스프리(주)(01042711) - 1분기 수집 완료

📥 2023년 반기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 반기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 반기 수집 완료
⚠️ 주식회사 엘지씨엔에스(00139834) - 반기 재무 데이터 없음
✅ 한국철강 주식회사(00687711) - 반기 수집 완료
✅ 대원강업 주식회사(00111847) - 반기 수집 완료
✅ 키스코홀딩스 주식회사(00159740) - 반기 수집 완료
✅ (주)팜젠사이언스(00135050) - 반기 수집 완료
✅ (주)지니뮤직(00223762) - 반기 수집 완료
✅ 새론오토모티브(주)(00305668) - 반기 수집 완료
✅ 아이에스동서(주)(00115977) - 반기 수집 완료
✅ 한온시스템주식회사(00161125) - 반기 수집 완료
✅ 글로벌텍스프리(주)(01042711) - 반기 수집 완료

📥 2023년 3분기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 3분기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 3분기 수집 완료
✅ 주식회사 엘지씨엔에스(00139834) - 3

In [25]:
import requests
import pandas as pd
import time

API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 여기에 API 키를 입력하세요
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

file_path = "/content/drive/MyDrive/Working/data/Listed_com_fss_test.xlsx"
corporations_df = pd.read_excel(file_path, sheet_name="lists", dtype={'code': str}, engine='openpyxl')

target_account_ids = [
    "ifrs-full_Assets",
    "ifrs-full_CurrentAssets",
    "ifrs-full_NoncurrentAssets",
    "ifrs-full_Liabilities",
    "ifrs-full_CurrentLiabilities",
    "ifrs-full_NoncurrentLiabilities",
    "ifrs-full_Equity",
    "ifrs-full_RetainedEarnings",
    "ifrs-full_CostOfSales",
    "dart_TotalAssets",
    "dart_OperatingIncomeLoss",
    "ifrs-full_ProfitLoss",
    "ifrs-full_IncomeTaxExpenseContinuingOperations",
    "ifrs-full_ProfitLossFromContinuingOperations"
]

report_periods = [
    ("2024", "11013", "1분기"),
    ("2024", "11012", "반기"),
    ("2024", "11014", "3분기"),
]

all_data = []

for year, reprt_code, label in report_periods:
    print(f"\n📥 {year}년 {label} 보고서 수집 시작\n")

    for _, row in corporations_df.iterrows():
        corp_code = row['corp_code']
        corp_name = row.get('name', '알 수 없음')

        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': reprt_code,
            'fs_div': 'OFS'  # 개별 재무제표
        }

        try:
            response = requests.get(FS_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get('status') == '000' and 'list' in data:
                for item in data['list']:
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추출
                    account_name = item.get('account_nm', '데이터 없음').strip() # 계정과목명 추출

                    if account_id in target_account_ids:
                        all_data.append({
                            "기업 이름": corp_name,
                            "기업 코드": corp_code,
                            "연도": year,
                            "분기": label,
                            "계정과목명": account_name, # 계정과목명 저장
                            "account_id": account_id,  # account_id 저장
                            "금액": item.get('thstrm_amount', '데이터 없음'),
                            "전기 금액": item.get('frmtrm_amount', '데이터 없음'),
                            "전전기 금액": item.get('lfy_amount', '데이터 없음')
                        })

                print(f"✅ {corp_name}({corp_code}) - {label} 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - {label} 재무 데이터 없음")
        except requests.exceptions.RequestException as e:
            print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

        time.sleep(1)

# 결과 저장
df_q = pd.DataFrame(all_data)
output_excel = "기업_재무제표_분기별_2023년_L2.xlsx"
df_q.to_excel(output_excel, index=False)

print(f"\n📦 데이터 저장 완료: {output_excel}")


📥 2024년 1분기 보고서 수집 시작

⚠️ (주)파라다이스(171265) - 1분기 재무 데이터 없음
⚠️ 한국타이어앤테크놀로지(주)(937324) - 1분기 재무 데이터 없음
⚠️ 주식회사 엘지씨엔에스(139834) - 1분기 재무 데이터 없음


KeyboardInterrupt: 

In [28]:
import requests
import pandas as pd
import time

API_KEY = '85006d20472a255d5eae25ee265b5ed1b78d1386'  # 여기에 API 키를 입력하세요
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

file_path = "/content/drive/MyDrive/Working/data/Listed_com_fss_test.xlsx"
corporations_df = pd.read_excel(file_path, sheet_name="lists", dtype={'corp_code': str}, engine='openpyxl')

target_account_ids = [
    "ifrs-full_Assets",
    "ifrs-full_CurrentAssets",
    "ifrs-full_NoncurrentAssets",
    "ifrs-full_Liabilities",
    "ifrs-full_CurrentLiabilities",
    "ifrs-full_NoncurrentLiabilities",
    "ifrs-full_Equity",
    "ifrs-full_RetainedEarnings",
    "ifrs-full_CostOfSales",
    "dart_TotalAssets",
    "dart_OperatingIncomeLoss",
    "ifrs-full_ProfitLoss",
    "ifrs-full_IncomeTaxExpenseContinuingOperations",
    "ifrs-full_ProfitLossFromContinuingOperations"
]

report_periods = [
    ("2023", "11013", "1분기"),
    ("2023", "11012", "반기"),
    ("2023", "11014", "3분기"),
]

all_data = []

for year, reprt_code, label in report_periods:
    print(f"\n📥 {year}년 {label} 보고서 수집 시작\n")

    for _, row in corporations_df.iterrows():
        corp_code = row['corp_code']
        corp_name = row.get('name', '알 수 없음')

        params = {
            'crtfc_key': API_KEY,
            'corp_code': corp_code,
            'bsns_year': year,
            'reprt_code': reprt_code,
            'fs_div': 'OFS'  # 개별 재무제표
        }

        try:
            response = requests.get(FS_URL, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get('status') == '000' and 'list' in data:
                for item in data['list']:
                    account_name = item.get('account_nm', '데이터 없음').strip()
                    account_id = item.get('account_id', '데이터 없음')

                    if account_id in target_account_ids:
                        all_data.append({
                            "기업 이름": corp_name,
                            "기업 코드": corp_code,
                            "연도": year,
                            "분기": label,
                            "계정과목명": account_name,
                            "account_id": account_id,
                            "금액": item.get('thstrm_amount', '데이터 없음'),
                            "전기 금액": item.get('frmtrm_amount', '데이터 없음'),
                            "전전기 금액": item.get('lfy_amount', '데이터 없음')
                        })

                print(f"✅ {corp_name}({corp_code}) - {label} 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - {label} 재무 데이터 없음")
        except requests.exceptions.RequestException as e:
            print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

        time.sleep(1)

# 결과 저장
df_q = pd.DataFrame(all_data)
output_excel = "기업_재무제표_분기별_2023년_L2.xlsx"
df_q.to_excel(output_excel, index=False)

print(f"\n📦 데이터 저장 완료: {output_excel}")


📥 2023년 1분기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 1분기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 1분기 수집 완료
⚠️ 주식회사 엘지씨엔에스(00139834) - 1분기 재무 데이터 없음
✅ 한국철강 주식회사(00687711) - 1분기 수집 완료
✅ 대원강업 주식회사(00111847) - 1분기 수집 완료
✅ 키스코홀딩스 주식회사(00159740) - 1분기 수집 완료
✅ (주)팜젠사이언스(00135050) - 1분기 수집 완료
✅ (주)지니뮤직(00223762) - 1분기 수집 완료
✅ 새론오토모티브(주)(00305668) - 1분기 수집 완료
✅ 아이에스동서(주)(00115977) - 1분기 수집 완료
✅ 한온시스템주식회사(00161125) - 1분기 수집 완료
✅ 글로벌텍스프리(주)(01042711) - 1분기 수집 완료

📥 2023년 반기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 반기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 반기 수집 완료
⚠️ 주식회사 엘지씨엔에스(00139834) - 반기 재무 데이터 없음
✅ 한국철강 주식회사(00687711) - 반기 수집 완료
✅ 대원강업 주식회사(00111847) - 반기 수집 완료
✅ 키스코홀딩스 주식회사(00159740) - 반기 수집 완료
✅ (주)팜젠사이언스(00135050) - 반기 수집 완료
✅ (주)지니뮤직(00223762) - 반기 수집 완료
✅ 새론오토모티브(주)(00305668) - 반기 수집 완료
✅ 아이에스동서(주)(00115977) - 반기 수집 완료
✅ 한온시스템주식회사(00161125) - 반기 수집 완료
✅ 글로벌텍스프리(주)(01042711) - 반기 수집 완료

📥 2023년 3분기 보고서 수집 시작

✅ (주)파라다이스(00171265) - 3분기 수집 완료
✅ 한국타이어앤테크놀로지(주)(00937324) - 3분기 수집 완료
✅ 주식회사 엘지씨엔에스(00139834) - 3